In [1]:
experiments_dev = ["WFV1_unet", "WF1_unet_all", "WF1_unet_rgbiswir", "WF1_unet_rgbi",
            "WF1_hrnet_rgbbs32","WF2_unet","WF2_hrnetsmall_rgb", "WF2_Unet_unused_from_checkpoint","WF2_unet_full","MNDWI"]

experiments_dev = ["WFV1_unet",  "WF2_Unet_unused_from_checkpoint","WF2_unet_full","MNDWI", "WF2_unet_S2"]

In [2]:
from ml4floods.data import utils
from ml4floods.models.utils import metrics
import pandas as pd
from ml4floods.models.config_setup import load_json

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"X:\home\kike\json_creds\ml4cc-general-access_request_pays.json"

In [3]:
def get_metrics_percode(mets):
    iou_per_code = pd.DataFrame(metrics.group_confusion(mets["confusions"],mets["cems_code"], metrics.calculate_iou,
                                                        label_names=[f"IoU_{l}"for l in ["land", "water"]]))

    recall_per_code = pd.DataFrame(metrics.group_confusion(mets["confusions"],mets["cems_code"], 
                                                           metrics.calculate_recall,
                                                           label_names=[f"Recall_{l}"for l in ["land", "water"]]))

    join_data_per_code = pd.merge(recall_per_code,iou_per_code,on="code")
    join_data_per_code = join_data_per_code.set_index("code")
    join_data_per_code = join_data_per_code*100
    return join_data_per_code
    # print(f"Mean values across flood events: {join_data_per_code.mean(axis=0).to_dict()}")

In [4]:
import itertools

prod_dev = "0_DEV"
dl_name = "test"

experiments_stats = []

for e in experiments_dev:
    # Obtain band configuration and modrivedel
    #config = utils.read_json_from_gcp(f"gs://ml4cc_data_lake/{prod_dev}/2_Mart/2_MLModelMart/{e}/config.json")
    if e != 'MNDWI':
        config = load_json(rf"X:\home\kike\Projectes\ml4floods\2_MLModelMart\{e}/config.json")
        channels = config["model_params"]["hyperparameters"]["channel_configuration"]
        architecture = config["model_params"]["hyperparameters"]["model_type"]
    else:
        channels, architecture = 'MNDWI','MNDWI'
        
    
    for dl_name in ["val", "test"]:
        data = load_json(rf"X:\home\kike\Projectes\ml4floods\2_MLModelMart\{e}\{dl_name}_masked_clouds.json")

        # compute metrics per flood
        metrics_per_code = get_metrics_percode(data)
        mean_metrics_code = metrics_per_code.mean(axis=0).to_dict()


        experiments_stats.append({
            "global_recall" : data["recall"]["water"],
            "global_iou" : data["iou"]["water"],
            "mean_recall_perflood" : mean_metrics_code["Recall_water"],
            "mean_iou_perflood" : mean_metrics_code["IoU_water"],
            "experiment": e,
            "channels": channels,
            "architecture": architecture,
            "subset": dl_name
        })

experiments_stats = pd.DataFrame(experiments_stats)
experiments_stats.to_clipboard(excel = True, sep = '\t')
experiments_stats = experiments_stats.sort_values(by = ['subset','mean_iou_perflood'], ascending= False)

experiments_stats

,global_recall,global_iou,mean_recall_perflood,mean_iou_perflood,experiment,channels,architecture,subset
0,0.956507,0.865877,82.927154,68.276717,WFV1_unet,all,unet,val
2,0.976674,0.823882,94.762671,64.442928,WF2_Unet_unused_from_checkpoint,all,unet,val
4,0.966258,0.804590,91.493690,63.375609,WF2_unet_full,all,unet,val
6,0.944945,0.550022,78.047791,51.049060,MNDWI,MNDWI,MNDWI,val
8,0.963941,0.559197,86.473873,42.087272,WF2_unet_S2,all,unet,val
1,0.872683,0.790005,90.970303,82.193955,WFV1_unet,all,unet,test
3,0.953666,0.751552,97.501674,73.066336,WF2_Unet_unused_from_checkpoint,all,unet,test
7,0.855593,0.738398,85.874504,70.453967,MNDWI,MNDWI,MNDWI,test
5,0.970411,0.681366,95.577827,70.162889,WF2_unet_full,all,unet,test
9,0.901680,0.684652,92.198331,62.913915,WF2_unet_S2,all,unet,test


In [14]:
experiments_stats.to_clipboard()

experiments_stats.index = experiments_stats['experiment']
experiments_stats.to_clipboard()